In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import MiniBatchKMeans
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm


def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out, indices_out = [], [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]
        indices_c = np.where(y == label)[0]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        n_sample = max(1, n_sample)

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])
        indices_out.append(indices_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0), np.concatenate(indices_out, axis=0)


def compute_uncertainties(model, X_pool, batch_size=10000):
    def predict_batch(X_chunk):
        probs = model.predict_proba(X_chunk)
        return 1 - np.max(probs, axis=1)

    chunks = [X_pool[i:i + batch_size] for i in range(0, len(X_pool), batch_size)]
    uncertainty_chunks = Parallel(n_jobs=-1)(delayed(predict_batch)(chunk) for chunk in tqdm(chunks, desc="Computing uncertainties"))
    return np.concatenate(uncertainty_chunks)


def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.2, random_state=42, max_clusters_per_iter=1000, max_top_k=10000, batch_size=10000):
    rng = np.random.RandomState(seed=random_state)
    n_total = len(X_pool)
    n_select = max(1, int(n_total * fraction))
    selected_indices = []

    if hasattr(X_pool, 'toarray'):
        X_pool = X_pool.toarray()

    classes = np.unique(y_pool)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        if len(idx_c) > 0:
            selected_idx = rng.choice(idx_c, size=1)[0]
            selected_indices.append(selected_idx)

    uncertainties = compute_uncertainties(model, X_pool, batch_size)

    n_remaining = n_select - len(selected_indices)
    current_indices = np.setdiff1d(np.arange(n_total), selected_indices)

    pbar = tqdm(total=n_select, desc="Selecting samples (active learning)")
    pbar.update(len(selected_indices))

    while n_remaining > 0 and len(current_indices) > 0:
        n_clusters = min(max_clusters_per_iter, n_remaining, len(current_indices))
        if n_clusters == 0:
            break

        top_k = min(n_clusters * 5, len(current_indices) - 1, max_top_k)
        if top_k <= 0:
            break

        current_uncertainties = uncertainties[current_indices]
        top_indices = np.argpartition(-current_uncertainties, top_k)[:top_k]
        top_global_indices = current_indices[top_indices]
        X_top = X_pool[top_global_indices]

        n_clusters = min(n_clusters, len(X_top))
        if n_clusters == 0:
            break

        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=1000, max_iter=20, init='k-means++')
        kmeans.fit(X_top)

        # Vectorized distance computation
        X_sq = np.sum(X_top ** 2, axis=1, keepdims=True)
        C_sq = np.sum(kmeans.cluster_centers_ ** 2, axis=1)
        XC = X_top @ kmeans.cluster_centers_.T
        dists = np.sqrt(np.maximum(X_sq - 2 * XC + C_sq, 0))
        cluster_closest = np.argmin(dists, axis=0)

        selected_indices.extend(top_global_indices[cluster_closest])

        pbar.update(len(cluster_closest))
        
        mask = ~np.isin(current_indices, top_global_indices[cluster_closest])
        current_indices = current_indices[mask]
        n_remaining -= n_clusters
    pbar.close()

    selected_indices = np.array(selected_indices[:n_select])
    X_selected = X_pool[selected_indices]
    y_selected = y_pool[selected_indices]
    remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    selected_class_counts = np.bincount(y_selected, minlength=len(classes))
    missing_classes = classes[selected_class_counts == 0]
    if len(missing_classes) > 0:
        print(f"Warning: Missing classes {missing_classes}. Adding one sample per missing class.")
        for c in missing_classes:
            idx_c = np.where(y_pool == c)[0]
            if len(idx_c) > 0:
                selected_idx = rng.choice(idx_c, size=1)[0]
                selected_indices = np.append(selected_indices, selected_idx)
        selected_indices = selected_indices[:n_select]
        X_selected = X_pool[selected_indices]
        y_selected = y_pool[selected_indices]
        remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
        X_rest = X_pool[remaining_idx]
        y_rest = y_pool[remaining_idx]

    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=len(classes))}")

    return X_rest, X_selected, y_rest, y_selected


# ==== Usage ====
# Replace with your actual data
X_pool = x_test_adv_0_1
y_pool = y_label_0_1

X_train_init, y_train_init, _ = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X_train_init, y_train_init)

_, X_50, _, y_50 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.5, random_state=42
)


Selecting samples (active learning): 100%|██████████| 1074388/1074388 [26:20<00:00, 679.95it/s]


Selected samples (fraction=0.5): 1074388
Label distribution: [ 53394     14  55145  10796   1648 815745      4 135521    658   1434
     29]


In [11]:
X_50.shape

(1074388, 42)

In [12]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt50 = DecisionTreeClassifier()
# dt50.fit(X_50, y_50)

xgb
RF


RandomForestClassifier()

In [13]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [14]:
import numpy as np



base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 6 8] [    10      3 235752   2988]
(10, 42) (10,)
Save 2 to baseline_Def2.npy
(3, 42) (3,)
Save 3 to baseline_Def3.npy
(235752, 42) (235752,)
Save 6 to baseline_Def6.npy
(2988, 42) (2988,)
Save 8 to baseline_Def8.npy
Execution Time: 0.517811 seconds
(716259,)
[1 3 5 6 8 9] [440263 272717     67   3127     58     27]
(440263, 42) (440263,)
Save 1 to BIM_Def1.npy
(272717, 42) (272717,)
Save 3 to BIM_Def3.npy
(67, 42) (67,)
Save 5 to BIM_Def5.npy
(3127, 42) (3127,)
Save 6 to BIM_Def6.npy
(58, 42) (58,)
Save 8 to BIM_Def8.npy
(27, 42) (27,)
Save 9 to BIM_Def9.npy
Execution Time: 6.511629 seconds
(716259,)
[ 1  2  3  5  6  8  9 10] [443198      2 245708   7081  17106    321      5   2838]
(443198, 42) (443198,)
Save 1 to FGSM_Def1.npy
(2, 42) (2,)
Save 2 to FGSM_Def2.npy
(245708, 42) (245708,)
Save 3 to FGSM_Def3.npy
(7081, 42) (7081,)
Save 5 to FGSM_Def5.npy
(17106, 42) (17106,)
Save 6 to FGSM_Def6.npy
(321, 42) (321,)
Save 8 to FGSM_Def8.npy
(5, 42) (5,)
Save 9 to FGSM_Def9

In [15]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 4 5 6 8] [     9      8      1      1 234246   4488]
(9, 42) (9,)
Save 2 to baseline_Def2.npy
(8, 42) (8,)
Save 3 to baseline_Def3.npy
(1, 42) (1,)
Save 4 to baseline_Def4.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(234246, 42) (234246,)
Save 6 to baseline_Def6.npy
(4488, 42) (4488,)
Save 8 to baseline_Def8.npy
Execution Time: 5.533434 seconds
(716259,)
[ 1  3  5  6  8 10] [432882 269825   3008  10434    105      5]
(432882, 42) (432882,)
Save 1 to BIM_Def1.npy
(269825, 42) (269825,)
Save 3 to BIM_Def3.npy
(3008, 42) (3008,)
Save 5 to BIM_Def5.npy
(10434, 42) (10434,)
Save 6 to BIM_Def6.npy
(105, 42) (105,)
Save 8 to BIM_Def8.npy
(5, 42) (5,)
Save 10 to BIM_Def10.npy
Execution Time: 3.785527 seconds
(716259,)
[ 1  3  5  6  8  9 10] [  5476 274864   7898 425768    224     44   1985]
(5476, 42) (5476,)
Save 1 to FGSM_Def1.npy
(274864, 42) (274864,)
Save 3 to FGSM_Def3.npy
(7898, 42) (7898,)
Save 5 to FGSM_Def5.npy
(425768, 42) (425768,)
Save 6 to FGSM_Def6.npy
(224, 42) 

In [16]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt50.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [17]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning_BatchMode/WUSTL_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [18]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ActiveLearning_BatchMode.csv")

In [19]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,0,10,3,0,0,235752,0,2988,0,0,0,238753
1,XGB50,440263,0,272717,0,67,3127,0,58,27,0,0,716259
2,XGB50,443198,2,245708,0,7081,17106,0,321,5,2838,0,716259
3,XGB50,440263,0,272717,0,67,3127,0,58,27,0,0,716259
4,XGB50,130,0,6350,34,9375,688874,7,2347,126,9016,0,716259
5,XGB50,187276,0,378371,53,62,147766,0,2674,53,4,0,716259
6,XGB50,0,28,15,0,0,707242,0,8974,0,0,0,716259
7,XGB50,14280,0,3578,38798,2484,61191,0,594921,868,123,16,716259
8,XGB50,1807,0,710141,0,209,4102,0,0,0,0,0,716259
9,XGB50,429120,1,254311,7,4235,28553,0,24,4,4,0,716259


In [ ]:
print("a")